In [4]:
import pandas as pd
import tushare as ts
import jqdatasdk
import os
import numpy as np
import datetime
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [5]:
# Obtain Data
df_300 = pd.read_csv("000300_daily.csv", header=0, names=["DateTime", "Open", "Close", "High", "Low", "Volume", "Money"])
df_300.head()

,DateTime,Open,Close,High,Low,Volume,Money
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09


In [6]:
# 继续对CSI300_DataFrame进行预处理 计算出每日收盘价Close差价, 以及每日变化百分比
# Pre_Close --> 对应昨日收盘价
# Change    --> 两日收盘价之差 今日 - 昨日
# Pct_Change--> 每日收盘价相较于前一天变化百分比
df_300['Pre_Close'] = df_300['Close'].shift(1)
df_300['Pre_Close'].fillna(950, inplace=True)
df_300['Change'] = df_300['Close'] - df_300['Pre_Close']
df_300['Pct_Change'] = round((df_300['Change'] / df_300['Pre_Close']) * 100,2)

In [7]:
# 检查预处理结束的df_300 dataframe 
# data feed 准备完毕 进入策略部分
df_300

,DateTime,Open,Close,High,Low,Volume,Money,Pre_Close,Change,Pct_Change
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,950.00,-5.44,-0.57
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,944.56,3.31,0.35
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,947.87,4.85,0.51
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,952.72,-16.11,-1.69
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,936.61,16.15,1.72
...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2635.79,28.51,1.08
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2664.30,-9.90,-0.37
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2654.40,26.67,1.00
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2681.07,-75.96,-2.83
